In [1]:
from imputationLibrary import forwardFilling, hotDeck, meanImputation, movingAverage, splineInterpolation, separate, randomSampleImputation
from imputationLibrary.util import util
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from tsmoothie.smoother import *
import matplotlib.pyplot as plt

In [2]:
INPUT = 'output/\CompleteCovid'

In [3]:
df = pd.read_csv(r'output\/CompleteCovid.csv',index_col=0, parse_dates=True)
df_train = df.loc['2020-01-22 09:00:00':'2020-02-26 02:00:00'].dropna(how='all', axis=1) #Selecting training data
df_test = df.loc['2020-02-26 03:00:00':, df_train.columns] #Selecting test data
period=60

In [5]:
df_white_noise, df_seasonal, df_trended, df_trend_and_seasonal = separate.separate(df_train, period)
df_seasonal_train, df_seasonal_test = randomSampleImputation.input(df_train.loc[:, df_seasonal.columns], df_test.loc[:, df_seasonal.columns], sliding_window_size = 0.03)
df_white_noise_train, df_white_noise_test =  meanImputation.input(df_train.loc[:, df_white_noise.columns], df_test.loc[:, df_white_noise.columns])
df_trended_train, df_trended_test =  forwardFilling.input(df_train.loc[:, df_trended.columns], df_test.loc[:, df_trended.columns])
df_trend_and_seasonal_train, df_trend_and_seasonal_test =  splineInterpolation.input(df_train.loc[:, df_trend_and_seasonal.columns], df_test.loc[:, df_trend_and_seasonal.columns])
df_train = pd.concat([df_seasonal_train, df_white_noise_train, df_trended_train, df_trend_and_seasonal_train], axis=1, join='outer')
df_test = pd.concat([df_seasonal_test, df_white_noise_test, df_trended_test, df_trend_and_seasonal_test], axis=1, join='outer')


### USE KALMAN FILTER TO SMOOTH ALL DATA (ONLY VISUALIZATION PURPOSE) ###

smoother_train = KalmanSmoother(component='level_longseason', 
                          component_noise={'level':0.1, 'longseason':0.1}, 
                          n_longseasons=365)
smoother_train.smooth(df_train.T)

smoother_test = KalmanSmoother(component='level_longseason', 
                          component_noise={'level':0.1, 'longseason':0.1}, 
                          n_longseasons=365)
smoother_test.smooth(df_test.T)

df_train = pd.DataFrame(data = smoother_train.smooth_data.T, index = df_train.index, columns= df_train.columns)
df_test = pd.DataFrame(data = smoother_test.smooth_data.T, index = df_test.index, columns= df_test.columns)

df_train.to_csv(r'output\/CompleteCovidNatureKalmanTrain_0.csv', index = True)
df_test.to_csv(r'output\/CompleteCovidNatureKalmanTest_0.csv', index = True)

Could not determine nature of the feature Zhejiang_deaths and it will be assigned to a group randomly.
Could not determine nature of the feature Diamond Princess cruise ship_recovered and it will be assigned to a group randomly.
Could not determine nature of the feature Fujian_deaths and it will be assigned to a group randomly.
Could not determine nature of the feature Shaanxi_deaths and it will be assigned to a group randomly.
